In [2]:
from IPython.display import display, HTML
import ipywidgets as widgets
from ipywidgets import Layout
import json
import sys
import os
import subprocess

# Prompt Generation
Edit the File ``prompt_config.json`` to change the prompt generation parameters. The file contains the following parameters:


In [3]:
import src.genPrompt as genPrompt

def check_config(config_file):
    buttonRunGenPrompt.layout.border = '1px solid black'
    buttonCheckConfig.layout.border = '2px solid red'
    output.clear_output()
    with open(config_file) as f:
        config = json.load(f)
    with output:
        print(json.dumps(config, indent=4))

    # get count of prompts
    prompts = genPrompt.generate_prompts_from_json(config)
    promptCount = len(prompts)
    textPromptCount.value = str(promptCount)

def run_gen_prompt(config_file):
    output.clear_output()
    buttonRunGenPrompt.layout.border = '2px solid red'
    buttonCheckConfig.layout.border = '1px solid black'
    with output:
        # run genPrompt
        command = f"python ./src/genPrompt.py {config_file}"
        print(f"running: {command}")
        os.system(command)
        

textfield = widgets.Text(value='prompt_config.json', description='Config file:', disabled=False)

# check config button
buttonCheckConfig = widgets.Button(description="Check config")
buttonCheckConfig.on_click(lambda b: check_config(textfield.value))

# run button
buttonRunGenPrompt = widgets.Button(description="Run genPrompt")
buttonRunGenPrompt.on_click(lambda b: run_gen_prompt(textfield.value))

# prompt count
textPromptCountDesc = widgets.HTML(value="Prompt count (Images per Class):")
textPromptCount = widgets.HTML(value="0")
texPromptCountHbox = widgets.HBox([textPromptCountDesc, textPromptCount])

# output
heading = widgets.HTML(value="<h2>Output</h2>")
output = widgets.Output()
output.layout = {'border': '1px solid black'}

# run genPrompt container

buttonHbox = widgets.HBox([buttonCheckConfig, buttonRunGenPrompt])
display(textfield, buttonHbox, texPromptCountHbox, heading ,output)

Text(value='prompt_config.json', description='Config file:')

HTML(value='<h2>Output</h2>')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

# Dataset Generation

In [4]:
from IPython.display import display, HTML
import ipywidgets as widgets

checkboxStates = {
    'Generate Dataset': False,
    'Segmentation': False,
    'Dataset': False,
    'CNN Training': False
}
def on_run_button_clicked(b):
    # clear print output
    # print the selected values
    output.clear_output()
    with output:
        print("Selected values:")
        print(f"  Dataset Name: {datasetName.value}")
        print(f"  Prompt Config: {promptConfig.value}")
        print(f"  {checkboxStates}")

        # get checkbox values
        checkboxArguments = ""
        for key, value in checkboxStates.items():
            if key == 'Generate Dataset':
                if value:
                    checkboxArguments += "--generateDataset True "
                else:
                    checkboxArguments += "--generateDataset False "
            elif key == 'Segmentation':
                if value:
                    checkboxArguments += "--segmentateImages True "
                else:
                    checkboxArguments += "--segmentateImages False "
            elif key == 'prepare Trainings Dataset':
                if value:
                    checkboxArguments += "--createTrainingDataset True "
                else:
                    checkboxArguments += "--createTrainingDataset False "
        # run script
        command = f"start cmd /k python generateDataset.py --datasetName {datasetName.value} --promptConfig {promptConfig.value} {checkboxArguments}"
        print(f"\nExecuting command: {command}")
        os.system(command)

def on_checkbox_change(change):
    checkboxStates[change.owner.description] = change.new

# Create a text widget for datasetName
datasetName = widgets.Text(
    value='urinal_Dataset_v1',
    description='Dataset Name:',
    disabled=False,
    layout=Layout(width='auto')  # Set the width to 'auto'
)
# Create a text widget for promptConfig
promptConfig = widgets.Text(
    value='prompt_config.json',
    description='Prompt Config:',
    disabled=False,
    layout=Layout(width='auto')  # Set the width to 'auto'
)
# combine the two text widgets into a VBox
textWidgets = widgets.VBox([datasetName, promptConfig])

# select multiple values
checkboxGenerateDataset = widgets.Checkbox(value=False, description='Generate Dataset')
checkboxGenerateDataset.observe(on_checkbox_change, names='value')
checkboxSegmentation = widgets.Checkbox(value=False, description='Segmentation')
checkboxSegmentation.observe(on_checkbox_change, names='value')
checkboxDateset = widgets.Checkbox(value=False, description='prepare Trainings Dataset')
checkboxDateset.observe(on_checkbox_change, names='value')

checkboxCNNTraining = widgets.Checkbox(value=False, description='CNN Training', disabled=True)
checkboxCNNTraining.observe(on_checkbox_change, names='value')

checkboxes = widgets.VBox([checkboxGenerateDataset, checkboxSegmentation, checkboxDateset, checkboxCNNTraining])

# Create a button widget to run code
button = widgets.Button(description="Run", button_style='success')
button.on_click(on_run_button_clicked)

# combine the checkboxes and button into a HBox
hbox = widgets.HBox([textWidgets, checkboxes, button])


# create output widget
outputTitle = widgets.HTML(value="<h2>Output</h2>")
output = widgets.Output()
output.layout = {
    'border': '1px solid black',
    'overflow_y': 'auto',  # Add a vertical scrollbar in case of overflow
}
outputVbox = widgets.VBox([outputTitle, output])

display(hbox, outputVbox)


# CNN Model Training

In [5]:
import ipywidgets as widgets
from IPython.display import display
import subprocess

def get_datasets_from_folder(folder):
    return [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

def on_run_button_clicked(b):

    output.clear_output()
    
    # Get widget values
    dataset = dataset_widget.value
    model = model_widget.value
    model_selection = model_selection_widget.value
    if model_selection == 'fine-tuning':
        model_selection = '1'
    else:
        model_selection = '2'

    # run script
    with output:
        command = f"start cmd /k python ./src/trainCNN/trainCNN.py --dataset {dataset} --model {model} --modelSelection {model_selection}"
        print(f"\nExecuting command: {command}")
        os.system(command)

# Create widgets
#dataset_widget = widgets.Text(description='Dataset', value='urinal_v2')
dataset_widget = widgets.Dropdown(description='Dataset', options=get_datasets_from_folder('./datasets'))
model_widget = widgets.Dropdown(description='Model', options=['inception_v3', 'resnet50'])
model_selection_widget = widgets.Dropdown(description='Model Selection', options=['fine-tuning', 'feature-extraction'])

run_button = widgets.Button(description="Run", button_style='success')

outputTitle = widgets.HTML(value="<h2>Output</h2>")
output = widgets.Output()
output.layout = {
    'border': '1px solid black',
    'overflow_y': 'auto',  # Add a vertical scrollbar in case of overflow
}

# Display widgets
display(dataset_widget, model_widget, model_selection_widget, run_button, outputTitle, output)

run_button.on_click(on_run_button_clicked)



Dropdown(description='Dataset', options=('urinal_Dataset_v12', 'urinal_Dataset_v12_seg', 'urinal_Dataset_v13',…

Dropdown(description='Model', options=('inception_v3', 'resnet50'), value='inception_v3')

Dropdown(description='Model Selection', options=('fine-tuning', 'feature-extraction'), value='fine-tuning')

Button(button_style='success', description='Run', style=ButtonStyle())

HTML(value='<h2>Output</h2>')

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…